### What is cross-validation?

In **cross-validation**, we run our modeling process on different subset of the data to get multiple measures of model quality.

### When should you use cross-validation?

Cross-validation gives a more accurate measure of model quality, which is especilly import if you are making a lot of modeling decisions. however, it can take longer to run, because it estimates multiple models (one of each fold)

So, give these tradeoffs, when should you see each approach?

 - For small datasets, where extra computation burden isn't a big deal, you should run cross-validation
 - for larger datasets, a single validation set is sufficient. Your code will run faster, and you may have enough data that there's little need to re-use some of it for holdout.
 
There's no simple threshould for what constitutes a large vs. samll dataset. But if your model takes a couple minutes or less to run, it's probably worth switching to cross-validation.

Alternative, you can run cross-validation and see if the scores for each experiment seem close. If each experiment yields the sanme results, a single validaton set is probably sufficient.